In [3]:
a = recording()

finish


In [7]:
data_directory = "C:/Users/user/desktop/sound_data/attendence/"

def read_file(read_file):
    return librosa.load(read_file)

array([[ 1.5258789e-04,  1.5258789e-04],
       [ 9.1552734e-05,  1.5258789e-04],
       [ 6.1035156e-05, -3.0517578e-05],
       ...,
       [ 6.1035156e-05, -1.2207031e-04],
       [ 9.1552734e-05, -6.1035156e-05],
       [ 0.0000000e+00,  3.0517578e-05]], dtype=float32)

In [2]:
import sounddevice as sd
import wave
import os
import numpy as np

data_directory = "C:/Users/user/desktop/sound_data/attendence/"
        
# 출석을 5번 말해 주세요 (5초 녹음)
def recording(duration = 5,fs = 44100,test_save=False):
    myrecording = sd.rec(int(duration * fs), samplerate=fs, channels=2)
    sd.wait()
    print("finish")
    if test_save==True:
        save_the_record(myrecording,'./recording_test.wav',sr=fs)
    
    return myrecording

# 녹음한 내용 듣기
def playsound(recorded_data,fs=44100):
    sd.play(recorded_data,samplerate=fs)
    sd.wait()

# 저장하기
def save_the_record(a,file_name,sr=44100,overwrite=False):
    if (overwrite == True) or (not os.path.exists(file_name)):
        with wave.open(data_directory+'sound_storage/'+file_name,'wb') as f:
            f.setnchannels(2)
            f.setsampwidth(2)
            f.setframerate(sr)
            f.writeframes((a*(2**15-1)).astype(np.int16))
    else:
        print("File Exists -> to overwrite the file change parameter overwirte=True")
        raise 

In [493]:
import librosa
import os,csv
from sklearn.preprocessing import StandardScaler
import numpy as np

data_directory = "C:/Users/user/desktop/sound_data/attendence/"

def read_file(read_file):
    return librosa.load(read_file)

def feature_extract_save(y,sr,save_file=None,label="Unknown"):
    
    # 한줄 생성
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    to_append = [np.mean(spec_cent),np.mean(spec_bw),np.mean(rolloff),np.mean(zcr),np.mean(chroma_stft)]    
    for e in mfcc:
        to_append.append(np.mean(e))
    to_append.append(label)
    
    # 저장유무체크 
    if save_file == False or save_file == None:
        pass
    else:
        # header생성
        if save_file == True:
            save_file = data_directory +'attend.csv'
        header = ['spec_cent','spec_bw','rolloff','zcr','chroma_stft']
        for i in range(20):
            header.append(f'mfcc{i}')
        header.append('label')
        
        # 파일유무체크
        if not os.path.exists(save_file):
            with open(save_file,'w',newline='') as f:
                writer = csv.writer(f)
                writer.writerow(header)

        with open(save_file, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(to_append)
            
    return np.array(to_append[0:-1])

In [481]:
import pandas as pd
from keras import models,layers
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier



# csv_data = 'C:/Users/user/desktop/sound_data/attendence/attend.csv'
def data_preprocessing(csv_data):
    data = pd.read_csv(csv_data,encoding='euc-kr')

    person_label = data[['label']]
    onehotencoder = OneHotEncoder()
    y = onehotencoder.fit_transform(person_label).toarray()
    
    # normalizing
    scaler = StandardScaler()
    X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
    return X,y,onehotencoder

def one_hot_decoder(oh_encoder,y):
    return oh_encoder.categories_[0][np.argmax(y,axis=1)].reshape(-1,1)

# spliting of dataset into train and test dataset
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

def model_making(input_shape,output_len):
    # creating a model
    model = models.Sequential()
    model.add(layers.Dense(256, activation='relu', input_shape=(input_shape[1:]) ))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(output_len, activation='softmax'))

    model.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])
    
    return model

def ML_model_making(X,y):
    MLmodel = GradientBoostingClassifier()
    MLmodel.fit(X,y)
    return MLmodel

def model_fitting(model,X,y):
    
#     X_train,X_test,y_train,y_test = train_test_split(X,y)
    
    history = model.fit(X, y, epochs=10, batch_size=128)
#     history = model.fit(X_train, y_train, epochs=10, batch_size=128)
    
#     test_loss, test_acc = model.evaluate(X_test,y_test)
#     print('test_acc: ',test_acc)
    return model
    
def model_predict(model,X):

    # predictions
    predictions = model.predict(X)

    return predictions

In [ ]:
## 기능구현

### 1. data gathering
* sound = recording()
* save_the_record(sound)
* data = feature_extract_save(sound)

### 1. modeling
* X,y,onehotencoder = data_preprocessing(csv_file)
* model = model_making(X.shape,y.shape[1])
* model = model_fitting(model,X,y)

### 1. predict_model
* 1. data gathering
* model_predict(model,onehotencoder,data)
* 

## 추가구현
* 단어인식(1d conv)

## 작동함수

In [482]:
# 음성 -> 모델
def gather_voice(name):
    # 출석 5번이상 말하기 (5초녹음)
    sound = recording()
    playsound(sound)
    save_the_record(sound,name+".wav")

    # 특징추출 후 저장
    features = feature_extract_save(np.mean(sound,axis=1),44100,save_file=True,label = name)

    # data로 model 학습
    csv_data = 'C:/Users/user/desktop/sound_data/attendence/attend.csv'
    X,y,oh_encoder = data_preprocessing(csv_data)
    model = model_making(X.shape,len(y))
    model = model_fitting(model,X,y)

    # 모델 저장
    return model,oh_encoder

def gather_voice_from_data(file,name):
    sound,sr = librosa.load(file)
    features = feature_extract_save(sound,44100,save_file=True,label = name)
    
    csv_data = 'C:/Users/user/desktop/sound_data/attendence/attend.csv'
    X,y,oh_encoder = data_preprocessing(csv_data)
    model = model_making(X.shape,y.shape[1])
    model = model_fitting(model,X,y)

    # 모델 저장
    return model,oh_encoder

# dir = "C:/Users/user/desktop/sound_data/attendence/sound_storage/" 
def make_model_from_dir(dir):
    for file in os.listdir(dir):
        sound,sr = librosa.load(dir+file)
        features = feature_extract_save(sound,44100,save_file=False,label = file.split(".")[0])
        
    csv_data = 'C:/Users/user/desktop/sound_data/attendence/attend.csv'
    X,y,oh_encoder = data_preprocessing(csv_data)
    model = model_making(X.shape,y.shape[1])
    model = model_fitting(model,X,y)

    # 모델 저장
    return model,oh_encoder

def MLmodel():
    csv_data = 'C:/Users/user/desktop/sound_data/attendence/attend.csv'
    X,y,oh_encoder = data_preprocessing(csv_data)
    y = one_hot_decoder(oh_encoder,y)
    return ML_model_making(X,y)

In [442]:
def predict(model,oh_encoder):
    # 1초 녹음후 예측
    person_X = recording(duration = 1.7)
#     playsound(person_X)
    # 모델 불러오기
    person_X_feature = feature_extract_save(person_X.mean(axis=1),44100)
    predict = model_predict(model,person_X_feature.reshape(1,25))
    
    # predict 가 누구인지
    return one_hot_decoder(oh_encoder,predict),predict

def MLpredict(model):
    # 1초 녹음후 예측
    person_X = recording(duration = 1.7)

    person_X_feature = feature_extract_save(person_X.mean(axis=1),44100)
    predict = model_predict(model,person_X_feature.reshape(1,25))
    
    # predict 가 누구인지
    return predict

## directory 로 부터 model 생성

In [494]:
dir = "C:/Users/user/desktop/sound_data/attendence/sound_storage/"
model,oh_encoder = make_model_from_dir(dir)

C:\Users\User\Anaconda3\envs\pro\lib\site-packages\librosa\core\audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


Epoch 1/10
6/6 [==============================] - 0s 63ms/step - loss: 1.8143 - accuracy: 0.1667
Epoch 2/10
6/6 [==============================] - 0s 667us/step - loss: 1.6026 - accuracy: 0.5000
Epoch 3/10
6/6 [==============================] - 0s 667us/step - loss: 1.4306 - accuracy: 0.8333
Epoch 4/10
6/6 [==============================] - 0s 666us/step - loss: 1.2824 - accuracy: 1.0000
Epoch 5/10
6/6 [==============================] - 0s 667us/step - loss: 1.1490 - accuracy: 1.0000
Epoch 6/10
6/6 [==============================] - 0s 501us/step - loss: 1.0256 - accuracy: 1.0000
Epoch 7/10
6/6 [==============================] - 0s 500us/step - loss: 0.9116 - accuracy: 1.0000
Epoch 8/10
6/6 [==============================] - 0s 500us/step - loss: 0.8055 - accuracy: 1.0000
Epoch 9/10
6/6 [==============================] - 0s 667us/step - loss: 0.7063 - accuracy: 1.0000
Epoch 10/10
6/6 [==============================] - 0s 665us/step - loss: 0.6154 - accuracy: 1.0000


In [507]:
predict(model,oh_encoder)

finish


(array([['용한인사']], dtype=object),
 array([[0., 0., 0., 1., 0., 0.]], dtype=float32))

In [500]:
ML = MLmodel()
MLpredict(ML)

C:\Users\User\Anaconda3\envs\pro\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


finish


array(['배강민'], dtype=object)

# 등록

In [ ]:
name = "cho2"
# 이름 입력하고 안녕하세요 3번 이상 말하기(5초 녹음)
model,oh_encoder = gather_voice(name)